In [1]:
import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras.layers as L
from sklearn.model_selection import KFold
import tensorflow.keras.backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import RootMeanSquaredError
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler
import os
import gc
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split

In [2]:
from google.colab import files
_ = files.upload()

Saving NO2+Altitude.zip to NO2+Altitude.zip


In [3]:
!unzip NO2+Altitude.zip -d "NO2+Altitude"
from IPython.display import clear_output
clear_output(wait=False)
!rm NO2+Altitude.zip

In [4]:
files = pd.read_csv("/content/NO2+Altitude/NO2+Altitude/files.csv")

In [5]:
files

,Date,SatFile,GroundFile,AltGrid,Fold
0,2020-05-20,sat_2020-05-20.npy,g_2020-05-20.npy,alt_2020-05-20.npy,1
1,2021-04-27,sat_2021-04-27.npy,g_2021-04-27.npy,alt_2021-04-27.npy,2
2,2020-03-03,sat_2020-03-03.npy,g_2020-03-03.npy,alt_2020-03-03.npy,0
3,2020-10-10,sat_2020-10-10.npy,g_2020-10-10.npy,alt_2020-10-10.npy,2
4,2020-07-17,sat_2020-07-17.npy,g_2020-07-17.npy,alt_2020-07-17.npy,1
...,...,...,...,...,...
481,2020-07-25,sat_2020-07-25.npy,g_2020-07-25.npy,alt_2020-07-25.npy,3
482,2020-08-22,sat_2020-08-22.npy,g_2020-08-22.npy,alt_2020-08-22.npy,2
483,2020-03-23,sat_2020-03-23.npy,g_2020-03-23.npy,alt_2020-03-23.npy,3
484,2020-05-17,sat_2020-05-17.npy,g_2020-05-17.npy,alt_2020-05-17.npy,4


In [6]:
def pearson(y_true,y_pred):
  if len(y_true.shape)!=1:
    true = []
    for i in range(y_true.shape[0]):
      true.extend(y_true[i])
    pred = []
    for i in range(y_pred.shape[0]):
      pred.extend(y_pred[i])
  else:
    true=y_true
    pred=y_pred
  return np.mean((np.array(true)-np.mean(true))*(np.array(pred)-np.mean(pred)))/(np.std(np.array(true))*np.std(np.array(pred)))

def pearsonCorrAvgDays(true,pred):
  # assert len(true.shape)>=3,"true must have at least 3 dimensions, found {}".format(len(true.shape))
  assert true.shape==pred.shape, "true and pred must have same shape, found {} and {}".format(true.shape,pred.shape)
  scores = []
  for i in range(true.shape[0]):
    scores.append(pearson(true[i],pred[i]))
  return np.mean(scores),scores

def pearsonCorrAvgPixels(true,pred):
  scores = []
  for i in range(true.shape[1]):
    scores.append(pearson(true[:,i],pred[:,i]))
  return np.mean(scores),scores

In [7]:
def loadData(df, alt=False, satdir="/content/NO2+Altitude/NO2+Altitude/satellite/", gdir="/content/NO2+Altitude/NO2+Altitude/ground/", altdir="/content/NO2+Altitude/NO2+Altitude/altitude/"):
    X_ = []
    X_alt = []
    Y = []
    for i in range(df.shape[0]):
        factor = 46 * (6.02214 / 6.023) * 1e2
        sat = factor * np.load(os.path.join(satdir, df["SatFile"].iloc[i]))
        alt = np.load(os.path.join(altdir, df["AltGrid"].iloc[i]))
        ground = np.load(os.path.join(gdir, df["GroundFile"].iloc[i])).flatten()
        if not np.isnan(np.sum(sat)) and not np.isnan(np.sum(ground)):
            X_.append(np.expand_dims(sat, axis=2))
            X_alt.append(np.expand_dims(alt, axis=2))
            Y.append(ground)
    return np.stack(X_, axis=0), np.stack(X_alt, axis=0), np.stack(Y, axis=0)


In [8]:
!pip install torch
!pip install vit-pytorch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.3/87.3 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 6.0 MB/s eta 0:00:00


In [20]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision.transforms import Resize
from sklearn.metrics import mean_squared_error, mean_absolute_error
from torchvision.models import vision_transformer
from vit_pytorch import ViT
from math import ceil

class CombinedModel(nn.Module):
    def __init__(self, sat_model, alt_model):
        super(CombinedModel, self).__init__()
        self.sat_model = sat_model
        self.alt_model = alt_model
        self.linear = nn.Linear(2 * 3283, 3283)

    def forward(self, sat_inputs, alt_inputs):
        sat_outputs = self.sat_model(sat_inputs)
        alt_outputs = self.alt_model(alt_inputs)
        combined_inputs = torch.cat((sat_outputs, alt_outputs), dim=1)
        outputs = self.linear(combined_inputs)
        return outputs


def build_model(X_train_sat, X_train_alt):
    # Build VIT model for satellite images
    sat_model = build_vit_model(X_train_sat)

    # Build VIT model for alternate images
    alt_model = build_vit_model(X_train_alt)

    # Create the combined model
    combined_model = CombinedModel(sat_model, alt_model)

    # Define the loss function
    criterion = nn.MSELoss()

    # Define the optimizer
    optimizer = torch.optim.Adam(combined_model.parameters(), lr=0.001)

    return combined_model, criterion, optimizer




def build_vit_model(X_train):
    # Calculate the image size and channels
    patch_size = 16  # Choose an appropriate patch size
    channels = X_train.shape[-1]

    image_height = X_train.shape[1]
    image_width = X_train.shape[2]

    resized_height = patch_size * (image_height // patch_size)
    resized_width = patch_size * (image_width // patch_size)

    assert resized_height > 0 and resized_width > 0, "Image dimensions are too small."

    model = ViT(
        image_size=(resized_height, resized_width),
        patch_size=patch_size,
        num_classes=3283,
        dim=64,
        depth=12,
        heads=8,
        mlp_dim=128,
        channels=channels
    )

    return model




In [22]:
import warnings
import numpy as np
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.optim import SGD
from vit_pytorch import ViT
from torchvision.transforms import RandomCrop, RandomHorizontalFlip, ColorJitter
from torchvision import transforms

# Model training
scores_list = []
rmses = []
maes = []
patch_size = 16
warnings.filterwarnings("ignore", category=UserWarning)

for fold in range(5):
    print("\nFold {}\n".format(fold))
    train_files = files[files["Fold"] != fold]
    val_files = files[files["Fold"] == fold]

    X_train_sat, X_train_alt, Y_train = loadData(train_files)  # Modify this line to load satellite and alternate images
    X_val_sat, X_val_alt, Y_val = loadData(val_files)  # Modify this line to load satellite and alternate images

    desired_height = 48
    desired_width = 64

    # Data augmentation for satellite images
    transform_sat = transforms.Compose([
        RandomCrop((desired_height, desired_width)),
        RandomHorizontalFlip(),
        ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4),
        Resize((desired_height, desired_width))
    ])

    # Data augmentation for alternate images
    transform_alt = transforms.Compose([
      RandomCrop((desired_height, desired_width)),
      RandomHorizontalFlip(),
      ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4),
      Resize((desired_height, desired_width))
    ])

    X_train_sat = transform_sat(torch.Tensor(X_train_sat).permute(0, 3, 1, 2)).permute(0, 2, 3, 1)
    X_val_sat = transform_sat(torch.Tensor(X_val_sat).permute(0, 3, 1, 2)).permute(0, 2, 3, 1)

    X_train_alt = transform_alt(torch.Tensor(X_train_alt).permute(0, 3, 1, 2)).permute(0, 2, 3, 1)
    X_val_alt = transform_alt(torch.Tensor(X_val_alt).permute(0, 3, 1, 2)).permute(0, 2, 3, 1)

    model, criterion, optimizer = build_model(X_train_sat, X_train_alt)
    optimizer = SGD(model.parameters(), lr=0.01, momentum=0.9)  # Example of using SGD optimizer

    if fold == 0:
        # print(model)
        pass

    train_dataset = torch.utils.data.TensorDataset(X_train_sat.permute(0, 3, 1, 2), X_train_alt.permute(0, 3, 1, 2),
                                                   torch.Tensor(Y_train))
    val_dataset = torch.utils.data.TensorDataset(X_val_sat.permute(0, 3, 1, 2), X_val_alt.permute(0, 3, 1, 2),
                                                 torch.Tensor(Y_val))

    train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=8)

    patience = 5  # Number of epochs to wait for improvement
    best_val_loss = np.inf
    num_epochs_without_improvement = 0

    # Learning rate scheduler
    lr_scheduler = ReduceLROnPlateau(optimizer, mode='min', patience=2, factor=0.1, verbose=True)

    for epoch in range(30):
        model.train()
        epoch_train_losses = []
        for sat_inputs, alt_inputs, targets in train_loader:
            optimizer.zero_grad()

            resized_sat_inputs = sat_inputs.clone()
            resized_alt_inputs = alt_inputs.clone()

            resized_sat_inputs[:, :, :desired_height, :desired_width] = transform_sat(
                sat_inputs[:, :, :desired_height, :desired_width])

            resized_alt_inputs[:, :, :desired_height, :desired_width] = transform_alt(
                alt_inputs[:, :, :desired_height, :desired_width])

            resized_width = patch_size * (resized_sat_inputs.shape[3] // patch_size)
            resized_sat_inputs = resized_sat_inputs[:, :, :, :resized_width]
            resized_alt_inputs = resized_alt_inputs[:, :, :, :resized_width]

            outputs = model(resized_sat_inputs, resized_alt_inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            epoch_train_losses.append(loss.item())

        model.eval()
        with torch.no_grad():
            val_outputs = []
            for sat_inputs, alt_inputs, targets in val_loader:
                outputs = model(sat_inputs, alt_inputs)
                val_outputs.append(outputs)
            val_outputs = torch.cat(val_outputs, dim=0)

        rmse = mean_squared_error(Y_val, val_outputs, squared=False)

        # Update learning rate
        lr_scheduler.step(rmse)

        # Check for early stopping
        if rmse < best_val_loss:
            best_val_loss = rmse
            num_epochs_without_improvement = 0
        else:
            num_epochs_without_improvement += 1
            if num_epochs_without_improvement == patience:
                print("Early stopping criteria met. Stopping training.")
                break

    rmse1 = mean_squared_error(Y_val, val_outputs, squared=False)
    rmses.append(rmse1)
    mae = mean_absolute_error(Y_val, val_outputs)
    maes.append(mae)

    print("Fold {} RMSE Score: {}".format(fold, rmse))
    print("Fold {} MAE Score: {}".format(fold, mae))
    #s, ls = pearsonCorrAvgDays(Y_val, val_outputs)
    #print("Fold {} Pearson coeff avg over days: {}".format(fold, np.mean([i for i in ls if not pd.isnull(i)])))
    #scores_list.append(ls)
    if fold != 4:
        del model
        _ = gc.collect()

print("\nCV RMSE Score: {}".format(np.mean(rmses)))
print("\nCV MAE Score: {}".format(np.mean(maes)))



Fold 0

Epoch 00005: reducing learning rate of group 0 to 1.0000e-03.
Epoch 00016: reducing learning rate of group 0 to 1.0000e-04.
Early stopping criteria met. Stopping training.
Fold 0 RMSE Score: 6.50958691951419
Fold 0 MAE Score: 5.061735013084693

Fold 1

Epoch 00009: reducing learning rate of group 0 to 1.0000e-03.
Epoch 00018: reducing learning rate of group 0 to 1.0000e-04.
Early stopping criteria met. Stopping training.
Fold 1 RMSE Score: 7.9011878561000275
Fold 1 MAE Score: 5.834527970146764

Fold 2

Epoch 00005: reducing learning rate of group 0 to 1.0000e-03.
Early stopping criteria met. Stopping training.
Fold 2 RMSE Score: 7.10519347376577
Fold 2 MAE Score: 5.4724845891320095

Fold 3

Epoch 00008: reducing learning rate of group 0 to 1.0000e-03.
Early stopping criteria met. Stopping training.
Fold 3 RMSE Score: 6.517686332832991
Fold 3 MAE Score: 4.954499973435546

Fold 4

Epoch 00005: reducing learning rate of group 0 to 1.0000e-03.
Early stopping criteria met. Stopping

In [24]:
true = Y_val[10].reshape((49,67))
pred = val_outputs[10].reshape((49,67))

In [25]:
import numpy as np
import folium
import branca.colormap as cm

ireland_bound = [-9.4003, -6.032, 51.828500000000005, 54.323]
llcrn, urcrn = (-9.4003, 51.8285), (-6.032, 54.323)

# Create latitude and longitude grids
num_cols = int((urcrn[0] - llcrn[0]) // 0.05)
num_rows = int((urcrn[1] - llcrn[1]) // 0.05)
lat_coords = np.linspace(llcrn[1], urcrn[1], num_rows+1)
lon_coords = np.linspace(llcrn[0], urcrn[0], num_cols+1)
lat,lon = np.meshgrid(lon_coords, lat_coords)

true = np.array(true)
# Initialize a map
m = folium.Map(location=[53.0, -7.5], zoom_start=7, tiles='Stamen Terrain')

# Convert the numpy array to a GeoJSON-like structure
features = []
for i in range(num_rows):
    for j in range(num_cols):
        polygon = [
            [lat[i, j], lon[i, j]],
            [lat[i+1, j], lon[i+1, j]],
            [lat[i+1, j+1], lon[i+1, j+1]],
            [lat[i, j+1], lon[i, j+1]],
            [lat[i, j], lon[i, j]]
        ]
        feature = {
            'type': 'Feature',
            'geometry': {
                'type': 'Polygon',
                'coordinates': [polygon]
            },
            'properties': {
                'value': true[i, j]  # Use the corresponding value from the 'true' array
            }
        }
        features.append(feature)

# Define a color scale for the 'true' values
vmin = np.min(true)
vmax = np.max(true)
colormap = cm.LinearColormap(colors=['blue', 'green', 'yellow', 'orange', 'red'], vmin=vmin, vmax=vmax)

# Add the GeoJSON layer to the map and set fill color based on 'true' values
folium.GeoJson(
    {
        'type': 'FeatureCollection',
        'features': features
    },
    style_function=lambda feature: {
        'fillColor': colormap(feature['properties']['value']),
        'color': 'none',
        'fillOpacity': 0.6
    }
).add_to(m)


# Create a color bar legend with values
legend_html = '''
    <div style="
        position: fixed;
        bottom: 360px; left: 520px; width: 70px; height: 134px;
        background-color: white;
        z-index: 9999; font-size: 12px;
        border: 2px solid grey; padding: 2px;
    ">

'''

num_values = 5  # Number of values to display in the color bar legend
value_range = np.linspace(vmin, vmax, num_values)

for value in value_range:
    color = colormap(value)
    label = f'{int(value)}'  # Format the value as desired
    legend_html += f'<p><span style="background-color:{color};">&nbsp;&nbsp;&nbsp;&nbsp;</span> {label}</p>'

legend_html += '</div>'

# Add the color bar legend to the map
m.get_root().html.add_child(folium.Element(legend_html))

# Crop the map to the bounds of the selected region
#m.fit_bounds([[llcrn[1], llcrn[0]], [urcrn[1], urcrn[0]]])
#m.fit_bounds(bounds)

# Save and display the map
m.save('map.html')

m

In [26]:
import numpy as np
import folium
import branca.colormap as cm

ireland_bound = [-9.4003, -6.032, 51.828500000000005, 54.323]
llcrn, urcrn = (-9.4003, 51.8285), (-6.032, 54.323)

# Create latitude and longitude grids
num_cols = int((urcrn[0] - llcrn[0]) // 0.05)
num_rows = int((urcrn[1] - llcrn[1]) // 0.05)
lat_coords = np.linspace(llcrn[1], urcrn[1], num_rows+1)
lon_coords = np.linspace(llcrn[0], urcrn[0], num_cols+1)
lat,lon = np.meshgrid(lon_coords, lat_coords)

pred = np.array(pred).astype(np.float64)
# Initialize a map
m = folium.Map(location=[53.0, -7.5], zoom_start=7, tiles='Stamen Terrain')

# Convert the numpy array to a GeoJSON-like structure
features = []
for i in range(num_rows):
    for j in range(num_cols):
        polygon = [
            [lat[i, j], lon[i, j]],
            [lat[i+1, j], lon[i+1, j]],
            [lat[i+1, j+1], lon[i+1, j+1]],
            [lat[i, j+1], lon[i, j+1]],
            [lat[i, j], lon[i, j]]
        ]
        feature = {
            'type': 'Feature',
            'geometry': {
                'type': 'Polygon',
                'coordinates': [polygon]
            },
            'properties': {
                'value': pred[i, j]  # Use the corresponding value from the 'pred' aklnxrray
            }
        }
        features.append(feature)

# Define a color scale for the 'pred' values
vmin = np.min(pred)
vmax = np.max(pred)
colormap = cm.LinearColormap(colors=['blue', 'green', 'yellow', 'orange', 'red'], vmin=vmin, vmax=vmax)

# Add the GeoJSON layer to the map and set fill color based on 'pred' values
folium.GeoJson(
    {
        'type': 'FeatureCollection',
        'features': features
    },
    style_function=lambda feature: {
        'fillColor': colormap(feature['properties']['value']),
        'color': 'none',
        'fillOpacity': 0.6
    }
).add_to(m)


# Create a color bar legend with values
legend_html = '''
    <div style="
        position: fixed;
        bottom: 300px; left: 520px; width: 70px; height: 134px;
        background-color: white;
        z-index: 9999; font-size: 12px;
        border: 2px solid grey; padding: 2px;
    ">

'''

num_values = 5  # Number of values to display in the color bar legend
value_range = np.linspace(vmin, vmax, num_values)

for value in value_range:
    color = colormap(value)
    label = f'{int(value)}'  # Format the value as desired
    legend_html += f'<p><span style="background-color:{color};">&nbsp;&nbsp;&nbsp;&nbsp;</span> {label}</p>'

legend_html += '</div>'

# Add the color bar legend to the map
m.get_root().html.add_child(folium.Element(legend_html))

# Crop the map to the bounds of the selected region
#m.fit_bounds([[llcrn[1], llcrn[0]], [urcrn[1], urcrn[0]]])
#m.fit_bounds(bounds)

# Save and display the map
m.save('map.html')

m